In [39]:
from collections import defaultdict

class Bucket:
    def __init__(self, zero_for_one):
        self.amt0 = 0
        self.amt1 = 0
        self.filled = False
        # TODO: remove? 
        self.zero_for_one = zero_for_one
        self.total_in = 0
        self.amt_ins = defaultdict(int)

    def inc(self, user, amt):
        if self.zero_for_one:
            assert self.amt1 == 0
            self.amt0 += amt
        else:
            assert self.amt0 == 0
            self.amt1 += amt
        self.total_in += amt
        self.amt_ins[user] += amt

    def dec(self, user, amt):
        assert self.amt_ins[user] >= amt
        if self.zero_for_one:
            assert self.amt1 == 0
            self.amt0 -= amt
        else:
            assert self.amt0 == 0
            self.amt1 -= amt
        self.total_in -= amt
        self.amt_ins[user] -= amt

    def __repr__(self):
        return f'filled: {self.filled} | zero_for_one: {self.zero_for_one} | amt0: {self.amt0} | amt1: {self.amt1} | total: {self.total_in} | {self.amt_ins}'

P1 = 1.01

class OrderBook:
    def __init__(self):
        self.tick = 0
        self.tick_spacing = 10
        self.dp = P1**self.tick_spacing
        self.p = 1
        # tick => current slot
        self.slots = defaultdict(int)
        # tick => slot => Bucket
        self.buckets = {}

    def swap(self, amt_in, min_amt_out, zero_for_one):
        # P1 = 1.0001
        # p = P1^tick = price of X (ETH) in terms of Y (USDC)
        # px = y

        #            Y | X
        #           1 for 0 -> + tick
        # - tick <- 0 for 1

        g = 0
        p = self.p
        t = self.tick
        amt_in_rem = amt_in
        amt_out = 0
        while amt_in_rem > 0:
            assert g < 100
            g += 1

            s = self.slots[t]
            bucket = self.buckets.get(t, {}).get(s, None)
            if bucket != None and bucket.total_in > 0:
                assert not bucket.filled

                d_in = 0
                d_out = 0
                # TODO: math func to calculate max in and out (px = y)
                if zero_for_one:
                    # y <- x
                    max_out = bucket.amt1
                    max_in = max_out / p
                    d_out = min(max_out, amt_in_rem * p)
                    d_in = min(max_in, amt_in_rem)
                    bucket.amt0 += d_in
                    bucket.amt1 -= d_out
                    assert bucket.amt1 >= 0
                else:
                    # y -> x
                    max_out = bucket.amt0
                    max_in = max_out * p
                    d_out = min(max_out, amt_in_rem / p)
                    d_in = min(max_in, amt_in_rem)
                    bucket.amt0 -= d_out
                    bucket.amt1 += d_in
                    assert bucket.amt0 >= 0

                amt_out += d_out
                amt_in_rem -= d_in

                assert amt_in_rem >= 0
            
                # TODO: efficient way to find the next tick
                if amt_in_rem > 0:
                    bucket.filled = True
                    self.slots[t] += 1
                    if zero_for_one:
                        assert bucket.amt0 > 0
                        assert bucket.amt1 == 0
                    else:
                        assert bucket.amt0 == 0
                        assert bucket.amt1 > 0
            if amt_in_rem > 0:
                if zero_for_one:
                    t -= self.tick_spacing
                    p /= self.dp    
                else:
                    t += self.tick_spacing
                    p *= self.dp
        self.p = p
        self.tick = t
        assert amt_out >= min_amt_out

        return amt_out
    
    def inc(self, tick, amt, zero_for_one, **kwargs):
        assert tick % self.tick_spacing == 0
        # TODO: allow placing order in current tick
        assert tick != self.tick
        
        msg_sender = kwargs["msg_sender"]
        if zero_for_one:
            assert tick > self.tick
        else:
            assert tick < self.tick
        
        s = self.slots[tick]
        if self.buckets.get(tick) is None:
            self.buckets[tick] = {}
        if self.buckets[tick].get(s) is None:
            self.buckets[tick][s] = Bucket(zero_for_one)

        bucket = self.buckets[tick][s]
        assert not bucket.filled
        bucket.inc(msg_sender, amt)

    def dec(self, tick, amt, **kwargs):
        # TODO: allow canceling order from current tick
        assert tick != self.tick
        
        msg_sender = kwargs["msg_sender"]
        s = self.slots[tick]
        bucket = self.buckets[tick][s]
        assert not bucket.filled
        bucket.dec(msg_sender, amt)

    def take(self, slot, tick, **kwargs):
        msg_sender = kwargs["msg_sender"]
        
        bucket = self.buckets[tick][slot]
        assert bucket.filled
        assert bucket.total_in > 0

        total_out = 0
        if bucket.zero_for_one:
            total_out = bucket.amt1
        else:
            total_out = bucket.amt0

        amt_out = total_out * bucket.amt_ins[msg_sender] / bucket.total_in
        bucket.amt_ins[msg_sender] = 0

        # TODO: check amt_out approx = P**tick * a_in
        # TODO: delete bucket if all taken
        return amt_out


In [44]:
book = OrderBook()
book.place(10, 100, True, msg_sender = "bob")
book.place(10, 100, True, msg_sender = "alice")
book.swap(500, 0, False)

for (t, sb) in book.buckets.items():
    print(t)
    for (s, b) in sb.items():
        print(s, b)

book.take(0, 10, msg_sender = "alice")

1 1 500 0
2 1.1046221254112045 500 0
3 1.2201900399479668 500 0
4 1.3478489153329056 500 0
5 1.4888637335882209 500 0
6 1.644631821843882 500 0
7 1.8166966985640904 500 0
8 2.006763368395384 500 0
9 2.216715217194257 500 0
10 2.44863267464848 500 0
11 2.7048138294215267 10.273465070303985 200
12 2.987797201097226 10.273465070303985 200
13 3.300386894573666 10.273465070303985 200
14 3.6456803861632476 10.273465070303985 200
15 4.027099216733587 10.273465070303985 200
16 4.448422896030052 10.273465070303985 200
17 4.913826354140581 10.273465070303985 200
18 5.4279213112123585 10.273465070303985 200
19 5.9958019753561675 10.273465070303985 200
20 6.623095521562628 10.273465070303985 200
21 7.31601785182994 10.273465070303985 200
22 8.081435189034703 10.273465070303985 200
23 8.926932114884414 10.273465070303985 200
24 9.86088672614516 10.273465070303985 200
25 10.892553653873602 10.273465070303985 200
26 12.03215576829744 10.273465070303985 200
27 13.290985478055402 10.273465070303985 200

244.863267464848